In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

from collections import Counter

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [45]:
# Explore keyword and location features
print("Keyword - missing:", train_df['keyword'].isna().sum(), f"({train_df['keyword'].isna().mean()*100:.1f}%)")
print("Keyword - unique:", train_df['keyword'].nunique())
print("\nLocation - missing:", train_df['location'].isna().sum(), f"({train_df['location'].isna().mean()*100:.1f}%)")
print("Location - unique:", train_df['location'].nunique())
print("\nTop keywords:")
print(train_df['keyword'].value_counts().head(10))

Keyword - missing: 61 (0.8%)
Keyword - unique: 221

Location - missing: 2533 (33.3%)
Location - unique: 3341

Top keywords:
keyword
fatalities     45
deluge         42
armageddon     42
damage         41
body%20bags    41
harm           41
sinking        41
evacuate       40
outbreak       40
fear           40
Name: count, dtype: int64


In [4]:
class TweetsDataset(Dataset):
    def __init__(self, df):
        self.labels = df['target'].tolist()
        self.text = df['text'].tolist()

        # Tokenizer
        self.tokenizer = get_tokenizer("basic_english")
        self.tokens = [self.tokenizer(t) for t in self.text]

        # Build vocab
        counter = Counter()
        for tokens in self.tokens:
            counter.update(tokens)
        self.vocab = Vocab(counter, specials=['<unk>', '<pad>'], min_freq=2)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        tokens = self.tokens[idx]
        indices = [self.vocab[token] for token in tokens]
        label = self.labels[idx]
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

    def print_vocab(self):
        print("Vocab size:", len(self.vocab))
        print("Top tokens:", self.vocab.itos[:10])

In [5]:
dataset = TweetsDataset(train_df)
len(dataset)
train_set, val_set = random_split(dataset, [.85, .15])
len(train_set), len(val_set)

(6472, 1141)

In [ ]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_size=64):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.hidden = nn.Linear(embed_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.hidden(embedded)
        return self.fc(x)

In [28]:
def collate_batch_bag(batch: list[tuple[torch.Tensor, torch.Tensor]]):

    # batch: list of (text_tensor_1d, label)
    text_list, label_list = [], []
    for text, label in batch:
        text_list.append(text)
        label_list.append(label)

    # Build offsets for EmbeddingBag
    lengths = torch.tensor([t.size(0) for t in text_list], dtype=torch.long)
    offsets = torch.cat((torch.tensor([0], dtype=torch.long), torch.cumsum(lengths, dim=0)[:-1]))
    text = torch.cat(text_list, dim=0)
    labels = torch.stack(label_list).long()

    # print(text, offsets, labels)
    return text, offsets, labels


def train(model, iterator, optimizer, criterion):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 100

    for idx, (text, offsets, labels) in enumerate(iterator):
        optimizer.zero_grad()
        text, offsets, labels = text.to(device), offsets.to(device), labels.to(device)
        output = model(text, offsets)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_acc += (output.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            print(f'At batch {idx}, accuracy: {total_acc/total_count:.4f}')
            total_acc, total_count = 0, 0


def evaluate(model, iterator, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for text, offsets, labels in iterator:
            text, offsets, labels = text.to(device), offsets.to(device), labels.to(device)
            output = model(text, offsets)
            loss = criterion(output, labels)
            total_acc += (output.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
    return total_acc / total_count


def predict(model, iterator):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text, offsets, labels in iterator:
            text, offsets = text.to(device), offsets.to(device)
            output = model(text, offsets)
            preds = output.argmax(1).cpu().numpy()
            predictions.extend(preds)
    return predictions

In [29]:
# train

LR = 0.01
BATCH_SIZE = 1

iterator = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch_bag
)

model = TextClassificationModel(vocab_size=len(dataset.vocab), embed_dim=64, num_class=2).to(device)
optimizer = optim.SGD(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(device)

train(model, iterator, optimizer, criterion)

evaluate_iterator = DataLoader(
    val_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_batch_bag
)

evaluate(model, evaluate_iterator, criterion)

At batch 100, accuracy: 0.4851
At batch 200, accuracy: 0.5500
At batch 300, accuracy: 0.4800
At batch 400, accuracy: 0.4900
At batch 500, accuracy: 0.5100
At batch 600, accuracy: 0.5600
At batch 700, accuracy: 0.6300
At batch 800, accuracy: 0.5300
At batch 900, accuracy: 0.5000
At batch 1000, accuracy: 0.5100
At batch 1100, accuracy: 0.6000
At batch 1200, accuracy: 0.4300
At batch 1300, accuracy: 0.7200
At batch 1400, accuracy: 0.5900
At batch 1500, accuracy: 0.6300
At batch 1600, accuracy: 0.6300
At batch 1700, accuracy: 0.5900
At batch 1800, accuracy: 0.5900
At batch 1900, accuracy: 0.5800
At batch 2000, accuracy: 0.5100
At batch 2100, accuracy: 0.6100
At batch 2200, accuracy: 0.6400
At batch 2300, accuracy: 0.6300
At batch 2400, accuracy: 0.5300
At batch 2500, accuracy: 0.5500
At batch 2600, accuracy: 0.5900
At batch 2700, accuracy: 0.5400
At batch 2800, accuracy: 0.6400
At batch 2900, accuracy: 0.5900
At batch 3000, accuracy: 0.5700
At batch 3100, accuracy: 0.6100
At batch 3200, ac

0.6389132340052586

In [19]:
# Create submission for test set

class TestDataset(Dataset):
    def __init__(self, df, vocab, tokenizer):
        self.text = df['text'].tolist()
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.tokens = [self.tokenizer(t) for t in self.text]
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        tokens = self.tokens[idx]
        indices = [self.vocab[token] for token in tokens]
        return torch.tensor(indices, dtype=torch.long), torch.tensor(0, dtype=torch.long)

# Create test dataset using the vocab from training
test_dataset = TestDataset(test_df, dataset.vocab, dataset.tokenizer)

# Create dataloader
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_batch_bag
)

# Generate predictions
predictions = predict(model, test_loader)

# Create submission dataframe
submission = pd.DataFrame({
    'id': test_df['id'],
    'target': predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print(f"Submission saved! Shape: {submission.shape}")
submission.head(10)

Submission saved! Shape: (3263, 2)


,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1
5,12,0
6,21,0
7,22,0
8,27,0
9,29,0


## Improved Model with Multi-Input Features

This enhanced version:
- Uses **keyword** and **location** as additional categorical features via embeddings
- Implements **LSTM** to capture word order (not just bag-of-words)
- Better handles missing values
- Improved training with Adam optimizer and learning rate scheduling

In [30]:
class ImprovedTweetsDataset(Dataset):
    """Dataset with keyword and location features + better text preprocessing."""
    
    def __init__(self, df, keyword_vocab=None, location_vocab=None, text_vocab=None, is_test=False):
        self.is_test = is_test
        if not is_test:
            self.labels = df['target'].tolist()
        
        self.text = df['text'].tolist()
        self.keywords = df['keyword'].fillna('<missing>').tolist()
        self.locations = df['location'].fillna('<missing>').tolist()
        
        # Tokenizer
        self.tokenizer = get_tokenizer("basic_english")
        self.tokens = [self.tokenizer(str(t).lower()) for t in self.text]
        
        # Build or reuse text vocab
        if text_vocab is None:
            counter = Counter()
            for tokens in self.tokens:
                counter.update(tokens)
            self.text_vocab = Vocab(counter, specials=['<unk>', '<pad>'], min_freq=2)
        else:
            self.text_vocab = text_vocab
        
        # Build or reuse keyword vocab
        if keyword_vocab is None:
            unique_keywords = list(set(self.keywords))
            self.keyword_vocab = {kw: idx for idx, kw in enumerate(unique_keywords)}
        else:
            self.keyword_vocab = keyword_vocab
        
        # Build or reuse location vocab (simplified - just use keyword approach)
        if location_vocab is None:
            # For locations, we'll simplify by hashing to reduce cardinality
            self.location_vocab = {}
            for loc in set(self.locations):
                if loc not in self.location_vocab:
                    self.location_vocab[loc] = len(self.location_vocab)
        else:
            self.location_vocab = location_vocab
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        # Text tokens
        tokens = self.tokens[idx]
        text_indices = [self.text_vocab[token] for token in tokens]
        
        # Keyword index
        keyword = self.keywords[idx]
        keyword_idx = self.keyword_vocab.get(keyword, 0)  # 0 for unknown
        
        # Location index
        location = self.locations[idx]
        location_idx = self.location_vocab.get(location, 0)  # 0 for unknown
        
        # Label
        label = self.labels[idx] if not self.is_test else 0
        
        return (
            torch.tensor(text_indices, dtype=torch.long),
            torch.tensor(keyword_idx, dtype=torch.long),
            torch.tensor(location_idx, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

In [31]:
class ImprovedModel(nn.Module):
    """Multi-input model with LSTM for text + embeddings for keyword/location."""
    
    def __init__(self, vocab_size, keyword_size, location_size, 
                 embed_dim=128, hidden_dim=128, num_class=2, dropout=0.3):
        super(ImprovedModel, self).__init__()
        
        # Text embedding + LSTM
        self.text_embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=2, 
                           batch_first=True, dropout=dropout, bidirectional=True)
        
        # Keyword and location embeddings
        self.keyword_embedding = nn.Embedding(keyword_size, 32)
        self.location_embedding = nn.Embedding(location_size, 32)
        
        # Classifier (concat LSTM output + keyword + location)
        combined_dim = hidden_dim * 2 + 32 + 32  # bidirectional LSTM + 2 embeddings
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(combined_dim, 64)
        self.fc2 = nn.Linear(64, num_class)
        
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.1
        self.text_embedding.weight.data.uniform_(-initrange, initrange)
        self.keyword_embedding.weight.data.uniform_(-initrange, initrange)
        self.location_embedding.weight.data.uniform_(-initrange, initrange)
    
    def forward(self, text, keyword, location):
        # Text: [batch, seq_len] -> embed -> LSTM
        embedded = self.text_embedding(text)  # [batch, seq, embed_dim]
        lstm_out, (hidden, _) = self.lstm(embedded)
        
        # Use final hidden states from both directions
        # hidden: [num_layers*2, batch, hidden_dim] for bidirectional
        hidden_fwd = hidden[-2, :, :]  # forward direction, last layer
        hidden_bwd = hidden[-1, :, :]  # backward direction, last layer
        lstm_repr = torch.cat([hidden_fwd, hidden_bwd], dim=1)  # [batch, hidden_dim*2]
        
        # Keyword and location embeddings
        keyword_emb = self.keyword_embedding(keyword)  # [batch, 32]
        location_emb = self.location_embedding(location)  # [batch, 32]
        
        # Concatenate all features
        combined = torch.cat([lstm_repr, keyword_emb, location_emb], dim=1)
        
        # Classifier
        x = self.dropout(combined)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

In [41]:
def collate_improved(batch):
    """Collate function for padded sequences + keyword/location."""
    text_list, keyword_list, location_list, label_list = [], [], [], []
    
    for text, keyword, location, label in batch:
        text_list.append(text)
        keyword_list.append(keyword)
        location_list.append(location)
        label_list.append(label)
    
    # Pad text sequences to same length
    from torch.nn.utils.rnn import pad_sequence
    text_padded = pad_sequence(text_list, batch_first=True, padding_value=0)
    
    keywords = torch.stack(keyword_list)
    locations = torch.stack(location_list)
    labels = torch.stack(label_list)
    
    return text_padded, keywords, locations, labels


def train_improved(model, iterator, optimizer, criterion, scheduler=None):
    """Training loop for improved model."""
    model.train()
    total_loss, total_acc, total_count = 0, 0, 0
    
    for text, keyword, location, labels in iterator:
        text = text.to(device)
        keyword = keyword.to(device)
        location = location.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        output = model(text, keyword, location)
        loss = criterion(output, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        total_acc += (output.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
    
    if scheduler:
        scheduler.step()
    
    return total_loss / len(iterator), total_acc / total_count


def evaluate_improved(model, iterator, criterion):
    """Evaluation loop for improved model."""
    model.eval()
    total_loss, total_acc, total_count = 0, 0, 0
    
    with torch.no_grad():
        for text, keyword, location, labels in iterator:
            text = text.to(device)
            keyword = keyword.to(device)
            location = location.to(device)
            labels = labels.to(device)
            
            output = model(text, keyword, location)
            loss = criterion(output, labels)
            
            total_loss += loss.item()
            total_acc += (output.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
    
    return total_loss / len(iterator), total_acc / total_count


def predict_improved(model, iterator):
    """Generate predictions for improved model."""
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for text, keyword, location, _ in iterator:
            text = text.to(device)
            keyword = keyword.to(device)
            location = location.to(device)
            
            output = model(text, keyword, location)
            preds = output.argmax(1).cpu().numpy()
            predictions.extend(preds)
    
    return predictions

In [42]:
# Create improved dataset
improved_dataset = ImprovedTweetsDataset(train_df)

print("Text vocab size:", len(improved_dataset.text_vocab))
print("Keyword vocab size:", len(improved_dataset.keyword_vocab))
print("Location vocab size:", len(improved_dataset.location_vocab))

# Split into train/val
train_set_improved, val_set_improved = random_split(improved_dataset, [0.85, 0.15])

# Create dataloaders
BATCH_SIZE_IMPROVED = 32

train_loader_improved = DataLoader(
    train_set_improved,
    batch_size=BATCH_SIZE_IMPROVED,
    shuffle=True,
    collate_fn=collate_improved
)

val_loader_improved = DataLoader(
    val_set_improved,
    batch_size=BATCH_SIZE_IMPROVED,
    shuffle=False,
    collate_fn=collate_improved
)

Text vocab size: 6982
Keyword vocab size: 222
Location vocab size: 3342


In [43]:
# Train improved model with better hyperparameters

# Initialize model
improved_model = ImprovedModel(
    vocab_size=len(improved_dataset.text_vocab),
    keyword_size=len(improved_dataset.keyword_vocab),
    location_size=len(improved_dataset.location_vocab),
    embed_dim=128,
    hidden_dim=128,
    num_class=2,
    dropout=0.3
).to(device)

# Better optimizer and scheduler
optimizer_improved = optim.Adam(improved_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer_improved, step_size=3, gamma=0.5)
criterion_improved = nn.CrossEntropyLoss()

# Training loop with validation monitoring
NUM_EPOCHS = 10
best_val_acc = 0

print("Training improved model...")
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_improved(
        improved_model, train_loader_improved, 
        optimizer_improved, criterion_improved, scheduler
    )
    val_loss, val_acc = evaluate_improved(
        improved_model, val_loader_improved, criterion_improved
    )
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(improved_model.state_dict(), 'best_model.pt')
        print(f"  ✓ New best model saved! (Val Acc: {val_acc:.4f})")

print(f"\nBest validation accuracy: {best_val_acc:.4f}")

Training improved model...
Epoch 1/10
  Train Loss: 0.5575, Train Acc: 0.7086
  Val Loss: 0.4389, Val Acc: 0.8028
  ✓ New best model saved! (Val Acc: 0.8028)
Epoch 2/10
  Train Loss: 0.3668, Train Acc: 0.8514
  Val Loss: 0.4568, Val Acc: 0.8019
Epoch 3/10
  Train Loss: 0.2765, Train Acc: 0.8971
  Val Loss: 0.4621, Val Acc: 0.7809
Epoch 4/10
  Train Loss: 0.1890, Train Acc: 0.9356
  Val Loss: 0.6138, Val Acc: 0.7862
Epoch 5/10
  Train Loss: 0.1378, Train Acc: 0.9527
  Val Loss: 0.6761, Val Acc: 0.7748
Epoch 6/10
  Train Loss: 0.1047, Train Acc: 0.9631
  Val Loss: 0.8079, Val Acc: 0.7713
Epoch 7/10
  Train Loss: 0.0677, Train Acc: 0.9742
  Val Loss: 0.9675, Val Acc: 0.7713
Epoch 8/10
  Train Loss: 0.0575, Train Acc: 0.9793
  Val Loss: 1.0492, Val Acc: 0.7695
Epoch 9/10
  Train Loss: 0.0488, Train Acc: 0.9816
  Val Loss: 1.2112, Val Acc: 0.7669
Epoch 10/10
  Train Loss: 0.0397, Train Acc: 0.9842
  Val Loss: 1.3019, Val Acc: 0.7651

Best validation accuracy: 0.8028


In [44]:
# Generate submission with improved model

# Load best model
improved_model.load_state_dict(torch.load('best_model.pt'))

# Create test dataset using training vocabs
test_dataset_improved = ImprovedTweetsDataset(
    test_df,
    keyword_vocab=improved_dataset.keyword_vocab,
    location_vocab=improved_dataset.location_vocab,
    text_vocab=improved_dataset.text_vocab,
    is_test=True
)

test_loader_improved = DataLoader(
    test_dataset_improved,
    batch_size=BATCH_SIZE_IMPROVED,
    shuffle=False,
    collate_fn=collate_improved
)

# Generate predictions
predictions_improved = predict_improved(improved_model, test_loader_improved)

# Create submission
submission_improved = pd.DataFrame({
    'id': test_df['id'],
    'target': predictions_improved
})

submission_improved.to_csv('submission_improved.csv', index=False)
print(f"Improved submission saved! Shape: {submission_improved.shape}")
print(f"\nPrediction distribution:")
print(submission_improved['target'].value_counts())
submission_improved.head(10)

Improved submission saved! Shape: (3263, 2)

Prediction distribution:
target
0    2201
1    1062
Name: count, dtype: int64


,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


## Summary of Improvements

### What Changed from Baseline (65% accuracy):

**1. Multi-Input Features** ✓
- Added **keyword** embeddings (32-dim)
- Added **location** embeddings (32-dim)
- Handles missing values with `<missing>` token

**2. Better Architecture** ✓
- **Bidirectional LSTM** (2 layers, 128 hidden units) instead of EmbeddingBag
- Captures word order and context (critical for "not bad" vs "bad")
- Dropout layers (0.3) to prevent overfitting
- Combined feature representation: LSTM output + keyword + location

**3. Training Improvements** ✓
- **Adam optimizer** (lr=0.001) instead of SGD
- **Learning rate scheduler** (decay every 3 epochs)
- **10 epochs** with validation monitoring
- **Gradient clipping** (prevents exploding gradients)
- **Model checkpointing** (saves best model based on validation)

**4. Data Improvements** ✓
- Text lowercasing for consistency
- Padded batches (variable length sequences)
- Larger batch size (32 vs 1-16)

### Expected Results:
- Baseline: ~65% (bag-of-words, text only)
- **Improved: 75-82%** (typical for LSTM + multi-features on this dataset)

### Next Steps to Push Further (if needed):
1. **Pre-trained embeddings**: Use GloVe or Word2Vec instead of random init
2. **Advanced models**: BERT, RoBERTa (transformers) - can reach 85%+
3. **Text preprocessing**: Remove URLs, handle hashtags, expand contractions
4. **Ensemble**: Combine multiple models
5. **Feature engineering**: Tweet length, punctuation count, capitalization patterns